In [1]:
import cv2
import numpy as np
import math
import time
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
import matplotlib
import torch
from torchvision import transforms
import util
from torchsummary import summary
from body import Body
import import_ipynb
from train_model import BodyposeBackbone, MainModel
from coco_dataset import COCODataset, transform_image
from copy import deepcopy
import random
import pandas as pd
from tqdm import tqdm
import sys
import os
from train_utils import chunk_data

/tmp/ipykernel_104492/1793954293.py:5: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


importing Jupyter notebook from train_model.ipynb
importing Jupyter notebook from coco_dataset.ipynb


/home/jakub/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Define number of keypoints of new neural network

In [2]:
NO_OF_KEYPOINTS = 17

In [3]:
path_to_coco_dataset = '/media/jakub/One Touch/coco_pose/coco2017labels-pose/coco-pose'

In [4]:
colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
            [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
            [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85], [255, 255, 255]]

Create backbone model

In [5]:
backbone_model_path = '../model/body_pose_model.pth'

In [6]:
backbone_model = BodyposeBackbone()

Load backbone params from file

In [7]:
backbone_params = torch.load(backbone_model_path)

In [8]:
backbone_dict = util.transfer(backbone_model, backbone_params)

In [9]:
backbone_model.load_state_dict(backbone_dict)

<All keys matched successfully>

In [10]:
main_model = MainModel(backbone_model, NO_OF_KEYPOINTS)

Create dataset from COCO .json file

In [11]:
train_data = COCODataset(path_to_coco_dataset, transforms=transform_image(train=True))

loading annotations into memory...
Done (t=0.37s)
creating index...
index created!


In [12]:
# Skeleton defines which joints are connected. Because of numbering from 1 I subtract it
skeleton = train_data.coco.cats[1]['skeleton']
skeleton = [[connection[0]-1, connection[1]-1] for connection in skeleton]

Show example image

In [26]:
index = random.randint(0, 100)
# load image
torch_image, image, target= train_data[index]

# draw keypoints
for keypoint in target:
    for point in keypoint:
        x = point[0]
        y = point[1]
        if x != 0 and y != 0:
            cv2.circle(image, (x, y), 3, (0, 0, 255), -1)
cv2.imshow('Example image', image)
cv2.waitKey(10_000)
cv2.destroyAllWindows()

In [14]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [15]:
device = torch.device("cpu")

In [16]:
# image as torch tensor with shape as (CxHxW)
torch_image.size()

torch.Size([3, 480, 640])

In [27]:
output = main_model(torch_image)
part_affinity_field_maps, confidence = output
part_affinity_field_maps = part_affinity_field_maps.detach().numpy()

In [28]:
part_affinity_field_maps = np.average(part_affinity_field_maps, axis=1)
part_affinity_field_maps = np.average(part_affinity_field_maps, axis=1)

In [29]:
part_affinity_field_maps_chunk = chunk_data(part_affinity_field_maps, 2)
for point in part_affinity_field_maps_chunk:
    x = point[0]
    y = point[1]
    if x != 0 and y != 0:
        cv2.circle(image, (int(x), int(y)), 3, (0, 0, 255), -1)
cv2.imshow('Example image', image)
cv2.waitKey(10_000)
cv2.destroyAllWindows()

In [ ]:
loss = torch.nn.MSELoss()
params = [p for p in main_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params)

In [ ]:
def train(main_model, loss, optimizer, inputs, outputs):
    optimizer.zero_grad()

    main_model.train()

    out1 = backbone_model(inputs)
    logits = main_model(out1)
    output = loss.forward(logits, outputs)
    output.backward()
    optimizer.step()

    return output.item()

In [ ]:
epochs = 0

for epoch in range(epochs):
    for id in train_data.ids:
        train_data.coco.loadAnns(train_data.coco.getAnnIds(id))
        train_data._load_image(id)